In [329]:
import numpy as np
import khet
import random
import pylab as plt
%load_ext line_profiler
%load_ext memory_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [2]:
from functools import lru_cache

## Alpha-beta pruning

In [304]:
from copy import deepcopy

def evaluate(board, player):
    """
    Evaluate the current state of the tic-tac-toe board for the given player.

    Parameters:
    - board: 3x3 list representing the tic-tac-toe board
    - player: The player for whom the evaluation is performed ('X' or 'O')

    Returns:
    - A numerical score indicating the desirability of the board for the player
    """

    # Check for a win
    if is_winner(tupleize(board), player):
        return 100  # Player wins, maximum score

    # Check for a loss
    opponent = 'O' if player == 'X' else 'X'
    if is_winner(tupleize(board), opponent):
        return -100  # Opponent wins, minimum score

    # Check for a draw
    if is_board_full(board):
        return 0  # Draw
    
    # return 0

    # No decisive outcome yet, evaluate based on potential for future wins
    player_score = evaluate_player_moves(board, player)
    opponent_score = evaluate_player_moves(board, opponent)

    return player_score - opponent_score

def is_game_over(board):
    return is_winner(tupleize(board), 'X') or is_winner(tupleize(board), 'O') or is_board_full(board)


@lru_cache(maxsize=None)
def is_winner(board, player):
    # Check rows, columns, and diagonals for a win
    nrows = len(board)
    
    return any(
        all(cell == player for cell in row) or  # Check rows
        all(board[i][j] == player for i in range(nrows)) or  # Check columns
        all(board[i][i] == player for i in range(nrows)) or  # Check main diagonal
        all(board[i][nrows - 1 - i] == player for i in range(nrows))  # Check secondary diagonal
        for j, row in enumerate(board)
    )

def _is_winner(board, player):
    # Check rows, columns, and diagonals for a win
    nrows = len(board)
    
    return any(
        all(cell == player for cell in row) or  # Check rows
        all(board[i][j] == player for i in range(nrows)) or  # Check columns
        all(board[i][i] == player for i in range(nrows)) or  # Check main diagonal
        all(board[i][nrows - 1 - i] == player for i in range(nrows))  # Check secondary diagonal
        for j, row in enumerate(board)
    )

def tupleize(x):
    """
    """
    return tuple([tuple(r) for r in x])


def is_board_full(board):
    # Check if the board is full (a draw)
    return all(all(cell for cell in row) for row in board)


def evaluate_player_moves(board, player):
    # Evaluate potential future wins for the player
    score = 0
    opponent = 'O' if player == 'X' else 'X'

    nrows = len(board)
    
    for i in range(nrows):
        for j in range(nrows):
            if board[i][j] is None:
                # Simulate making a move for the player
                board[i][j] = player

                # Check if the move leads to a win
                if is_winner(tupleize(board), player):
                    score += 1
                    
                elif is_winner(tupleize(board), opponent):
                    score -= 3

                # Undo the move for the next iteration
                board[i][j] = None

    return score

def make_move(board, move, color):
    """
    """
    bb = deepcopy(board)
    if bb[move[0]][move[1]] is None:
        bb[move[0]][move[1]] = color

    return bb

def get_all_valid_moves(board):
    moves = []
    nrows = len(board)
    for i in range(nrows):
        for j in range(nrows):
            if board[i][j] is None:
                moves.append((i, j))
                
    return moves

In [305]:
import math

transposition_table = {}
opponent_map = {'X': 'O', "O": "X"}

def alpha_beta_search(
    board, depth, alpha=-math.inf, beta=math.inf, maximizing_player=True, player='X'
):
    """
    Maximizing player is X here
    """    
    if player == 'X':
        max_player = 'X'
        min_player = 'O'
    else:
        max_player = 'O'
        min_player = 'X'
        
    if depth == 0 or is_game_over(board):
        return evaluate(board, max_player), (-1, -1)

    # Get all of the potential legal moves
    legal_moves = get_all_valid_moves(board)

    if maximizing_player:
        best_move = None
        max_eval = -math.inf
        for move in legal_moves:
            new_board = make_move(board, move, max_player)
            eval_score, _move = alpha_beta_search(
                new_board, depth - 1, alpha, beta, False, player=max_player
            )
            if eval_score > max_eval:
                max_eval = eval_score
                best_move = move
                
            alpha = max(alpha, eval_score)

            if beta <= alpha:
                break  # Beta cutoff

        return alpha, best_move

    else:
        best_move = None
        min_eval = math.inf
        for move in legal_moves:
            new_board = make_move(board, move, min_player)
            eval_score, _move = alpha_beta_search(
                new_board, depth - 1, alpha, beta, True, player=max_player
            )
            if eval_score < min_eval:
                min_eval = eval_score
                best_move = move
                
            beta = min(beta, min_eval)
            
            if beta <= alpha:
                break  # Alpha cutoff

        return beta, best_move
    
class Bot:
    def __init__(self, marker='X', mode='random'):
        """
        """
        assert marker in ['X', "O"]
        self.marker = marker
        assert mode in ['random', 'minimax']
        self.mode = mode
    
    def make_move(self, board, depth=5):
        """
        """
        if self.mode == 'minimax':
            _, move = alpha_beta_search(board, depth, maximizing_player=True, player=self.marker)
        else:
            all_moves = get_all_valid_moves(board)
            move = all_moves[random.randint(0, len(all_moves) - 1)]
            
        return move

In [306]:
def print_board(board):
    for row in board:
        print ("".join(['.' if i is None else i for i in row]))
    print ()

In [307]:
def play_game(
    player1_bot=True,
    bot1_type="minimax",
    depth1=5,
    player2_bot=True,
    bot2_type="random",
    depth2=5,
    verbose=True,
    first_move_random=True,
    marker1='X',
    marker2='O'
):
    """
    """
    player1 = Bot(mode=bot1_type, marker=marker1)
    player2 = Bot(mode=bot2_type, marker=marker2)
    nrows = 3

    board = [[None for j in range(nrows)] for i in range(nrows)]
    if verbose:
        print_board(board)

    boards = []

    ci = 0

    while not is_game_over(board):
        if ci == 0 and first_move_random:
            player1.mode = "random"
            move = player1.make_move(board, depth=depth1)
            player1.mode = bot1_type
        else:
            move = player1.make_move(board, depth=depth1)

        board = make_move(board, move, marker1)
        boards.append(board)
        if is_game_over(board):
            break

        move = player2.make_move(board, depth=depth2)
        board = make_move(board, move, marker2)
        boards.append(board)

        if verbose:
            print_board(board)

        ci += 1

    if verbose:
        print_board(board)
    return boards

## Minimax Algorithm w/ Pruning

In [203]:
ngames = 1000
games = []
for n in range(ngames):
    games.append(
        play_game(
            bot1_type="minimax",
            bot2_type="random",
            depth1=3,
            depth2=1,
            verbose=False,
            first_move_random=False,
            marker1='O',
            marker2='X'
        )[-1]
    )
    
wins = 0
losses = 0
ties = 0
for game in games:
    if is_winner(tupleize(game), 'O'):
        wins += 1
    elif is_winner(tupleize(game), 'X'):
        losses += 1
    else:
        ties += 1

In [202]:
print ("Minimax algorithm won", wins, "games, lost", losses, "games, and tied", ties, "games as first player")

Minimax algorithm won 991 games, lost 0 games, and tied 9 games as first player


In [210]:
ngames = 1000
games = []
for n in range(ngames):
    games.append(
        play_game(
            bot1_type="random",
            bot2_type="minimax",
            depth1=1,
            depth2=4,
            verbose=False,
            first_move_random=False,
            marker1='O',
            marker2='X'
        )[-1]
    )
    
wins = 0
losses = 0
ties = 0
for game in games:
    if is_winner(tupleize(game), 'X'):
        wins += 1
    elif is_winner(tupleize(game), 'O'):
        losses += 1
    else:
        ties += 1

In [211]:
print ("Minimax algorithm won", wins, "games, lost", losses, "games, and tied", ties, "games as second player")

Minimax algorithm won 816 games, lost 0 games, and tied 184 games as second player


In [ ]:
class MCTS:
    def __init__(self):
        """
        """
        return
    
    def search(state, game, nnet):
        """
        """
        if is_game_over(game):
            return 
        
        if s not in visited:
            visited.add(s)
            P[s], v = nnet.predict(s)
            return -v
        
        max_u, best_a = -float('inf'), -1
        for a in get_all_valid_moves(game):
            u = Q[s][a] + c_puct * P[s][a] * np.sqrt(sum(N[s])) / (1 + N[s][a])
            
            if u > max_u:
                max_u = u
                best_a = a
                
        a = best_a
        Q[s][a] = (N[s][a]*Q[s][a] + v) / (N[s][a] + 1)
        N[s][a] += 1
        return -v

## Neural Network

This approach uses reinforcement learning to train a neural network 

In [214]:
import torch
from torch import nn
from torch.utils.data import DataLoader

In [258]:
def loss_function(state, outcome, network, pi):
    """
    """
    logits = network(state)
    prob = nn.Softmax(dim=1)(logits[:, :-1])
    vs = logits[:, -1]
    return torch.square(outcome - vs).sum() - torch.sum(pi * torch.log(prob))

class NeuralNetwork(nn.Module):
    def __init__(self):
        """
        """
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(9, 45),
            nn.ReLU(),
            nn.Linear(45, 45),
            nn.ReLU(),
            nn.Linear(45, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

\begin{equation}
\mathcal{L} = \sum_t \left(v_\theta\left(s_t\right) - z_t\right)^2 - \vec{\pi}_t \cdot \log\left(\vec{p}_\theta\left(s_t\right)\right)
\end{equation}

In [398]:
did = {}
did[None] = 0
did["X"] = 1
did['O'] = -1

# Simulate games in this cell
ngames = 1000
games = []

for n in range(ngames):
    _here_games = play_game(
            bot1_type="minimax",
            bot2_type="random",
            depth1=3,
            depth2=1,
            verbose=False,
            first_move_random=True,
            marker1='X',
            marker2='O'
        )
    all_games = list(map(lambda game: sum(game, []), _here_games))
    all_games = [[did[square] for square in g] for g in all_games]
    games.append(all_games)
    #games += all_games
    
results = []
for game in games:
    g = game[-1]
    if _is_winner([g[:3], g[3:6], g[6:9]], 1):
        results += [1]
    elif _is_winner([g[:3], g[3:6], g[6:9]], -1):
        results += [-1]
    else:
        results += [0]
        
states = np.array(sum(games, []))
results = np.array(results)

In [ ]:
loss = loss_function(y_pred, ybatch)
optimizer.zero_grad()
loss.backward()
optimizer.step()

## Monte Carlo Tree Search

In [363]:
class TicTacToe:
    def __init__(self, nsquares=3):
        """
        """
        self.board = [[None for _ in range(nsquares)] for _ in range(nsquares)]
                
    def get_all_valid_moves(self):
        moves = []
        nrows = len(self.board)
        for i in range(nrows):
            for j in range(nrows):
                if self.board[i][j] is None:
                    moves.append((i, j))

        return moves
    
    def make_move(self, move, player):
        """
        """
        self.board[move[0]][move[1]] = player
    
    @property
    def state(self):
        """
        """
        return "".join([item if item else '.' for row in self.board for item in row])
    

def predict(game):
    """
    """
    moves = game.get_all_valid_moves()
    prob = np.random.uniform(0, 1, size=len(moves))
    prob /= prob.sum()
    return {m: p for m, p in zip(moves, prob)}, np.random.uniform(-1, 1)

In [413]:
def get_state_repr(game):
    """
    """
    return "".join([item if item else "." for row in game for item in row])


class MCTS:
    def __init__(self):
        """
        """
        self.Q = {}  # stores Q values for state and action
        self.Nsa = {}  # stores times state and action were visited
        self.P = {}  # stores initial policy
        self.N = {}  # stores times state was visited
        self.Es = {}  # stores whether state is terminal
        self.Vs = {}  # stores valid moves for state s
        self.visited = set()

    def get_action_prob(self, game, nsims=100):
        """
        """
        for _ in range(nsims):
            self.search(game)

        s = get_state_repr(game)
        counts = [
            self.Nsa[(s, a)] if (s, a) in self.Nsa else 0
            for a in get_all_valid_moves(game)
        ]
        
        counts_sum = np.sum(counts)
        probs = np.array(counts) / counts_sum
        return probs

    def search(self, game, c_puct=1):
        """
        Fix search function
        """
        board = game.board
        
        if is_game_over(board):
            return

        s = get_state_repr(board)
        
        if s not in self.Es:
            self.Es[s] = game

        # leaf node
        if s not in self.P:
            self.P[s], v = self.predict(board)
            valids = get_all_valid_moves(board)
            self.P[s] *= valids
            self.P[s] /= self.P[s].sum()
            self.Vs[s] = valids
            self.Ns[s] = 0
            return -v
        
        valids = self.Vs[s]
        current_best, best_action = -float('inf'), -1

        for a in get_all_valid_moves(game):
            if valids[a]:
                if (s, a) in self.Qsa:
                    u = self.Qsa[(s, a)] + c_puct * self.P[s][a] * np.sqrt(self.Ns[s]) / (1 + self.Nsa[(s, a)])
            else:
                u = c_puct * self.P[s][a] * np.sqrt(self.Ns[s] + 1e-4)

            if u > current_best:
                current_best = u
                best_action = a

        a = best_action
        make_move(game, player)
        
        if (s, a) in self.Qsa:
            self.Qsa[(s, a)] = (self.Nsa[(s, a)] * self.Qsa[(s, a)] + v) / (self.Nsa[(s, a)] + 1)
            self.Nsa[(s, a)] += 1
        else:
            self.Qsa[(s, a)] = v
            self.Nsa[(s, a)] = 1

        self.Ns[s] += 1
        return -v

In [414]:
game = TicTacToe()
get_state_repr(game.board)

'.........'

In [415]:
mcts = MCTS()
num_mcts_sims = 20

for _ in range(num_mcts_sims):
    mcts.search(game)

AttributeError: 'MCTS' object has no attribute 'predict'

In [391]:
sum(mcts.P['.........'].values())

1.0